In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from shash.shash_torch import Shash
import random

__author__ = "Randal J. Barnes and Elizabeth A. Barnes"
__date__ = "15 October 2021"

In [ ]:
SEED = 31415
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True

# Test the routines

In [ ]:
xmin = -3
xmax = 9
n = 1000

x = torch.linspace(xmin, xmax, n)

shash_params = torch.ones(size=(x.shape[0], 4))

shash_params[:, 0] = shash_params[:, 0]
shash_params[:, 1] = 2 * shash_params[:, 1]
shash_params[:, 2] = 0.5 * shash_params[:, 2]
shash_params[:, 3] = 1.5 * shash_params[:, 3]

## Test the pdf

In [ ]:
dist = Shash(shash_params)
f = dist.prob(x)
print(f"Total area under the pdf = {(xmax - xmin) * torch.mean(f, axis=-1)}")

plt.plot(x, f, "-")
plt.title("PDF")
plt.show()

## Compare prob to the tfp version.

In [ ]:
# tfp_shash = tfp.distributions.SinhArcsinh(
#     loc=mu, scale=sigma, skewness=gamma, tailweight=tau
# )
# f_tfp = tfp_shash.prob(x)

# plt.plot(x, f, "-", x, f_tfp, "-")
# plt.title("PDF")

## Test the cdf

In [ ]:
F = dist.cdf(x)

plt.plot(x, F, "-")
plt.title("CDF")
plt.show()

In [ ]:
xx = torch.linspace(1, 3, n)
f = dist.prob(xx)
F = dist.cdf(xx)

area_by_pdf = (3 - 1) * torch.mean(f, axis=-1)
area_by_cdf = F[-1] - F[0]

print(f"integral of the pdf from 1 to 3 = {area_by_pdf}")
print(f"cdf(3) - cdf(1) = {area_by_cdf}")
print(f"error = {area_by_pdf - area_by_cdf}")

## Test the quantile vs. the cdf

In [ ]:
pr = torch.linspace(0.0001, 0.9999, n)
xx = dist.quantile(pr)
F = dist.cdf(xx)

plt.plot(xx, F - pr, "-")
plt.title("CDF to quantiles inversion errors")

# Test the computed distribution properties

In [ ]:
NTEST = 100
NN = 100000

mu = tf.random.uniform(shape=[NTEST, 1], minval=-1, maxval=1)
sigma = tf.random.uniform(shape=[NTEST, 1], minval=1, maxval=2)
gamma = tf.random.uniform(shape=[NTEST, 1], minval=-1, maxval=1)
tau = tf.random.uniform(shape=[NTEST, 1], minval=1, maxval=2)

## Test the median by comparison with quantile at 0.5

In [ ]:
median_target = shash.quantile(0.5 * tf.ones(shape=[NTEST, 1]), mu, sigma, gamma, tau)
median_compute = shash.median(mu, sigma, gamma, tau)
error = median_compute - median_target

print(f"max median error = {tf.math.reduce_max(error)}")
print(f"min median error = {tf.math.reduce_min(error)}")

## Test the mean by numerical integration

In [ ]:
mean_target = np.zeros([NTEST, 1])

lb = shash.quantile(0.00001 * tf.ones(shape=[NTEST, 1]), mu, sigma, gamma, tau)
ub = shash.quantile(0.99999 * tf.ones(shape=[NTEST, 1]), mu, sigma, gamma, tau)

for i in range(NTEST):
    xx = tf.cast(tf.linspace(lb[i], ub[i], NN), dtype=tf.float32)
    ff = shash.prob(xx, mu[i], sigma[i], gamma[i], tau[i])
    mean_target[i] = tf.reduce_mean(xx * ff) * (ub[i] - lb[i])

mean_compute = shash.mean(mu, sigma, gamma, tau)
error = mean_compute - mean_target

print(f"max mean error = {tf.math.reduce_max(error):.6f}")
print(f"min mean error = {tf.math.reduce_min(error):.6f}")

## Test the standard deviation by numerical integration

In [ ]:
stddev_target = np.zeros([NTEST, 1])
mean_compute = shash.mean(mu, sigma, gamma, tau)

lb = shash.quantile(0.00001 * tf.ones(shape=[NTEST, 1]), mu, sigma, gamma, tau)
ub = shash.quantile(0.99999 * tf.ones(shape=[NTEST, 1]), mu, sigma, gamma, tau)

for i in range(NTEST):
    xx = tf.cast(tf.linspace(lb[i], ub[i], NN), dtype=tf.float32)
    zz = tf.math.square(xx - mean_compute[i])
    ff = shash.prob(xx, mu[i], sigma[i], gamma[i], tau[i])
    stddev_target[i] = tf.sqrt(tf.reduce_mean(zz * ff) * (ub[i] - lb[i]))

stddev_compute = shash.stddev(mu, sigma, gamma, tau)
error = stddev_compute - stddev_target

print(f"max stddev error = {tf.math.reduce_max(error):.6f}")
print(f"min stddev error = {tf.math.reduce_min(error):.6f}")

## Test the variance by numerical integration

In [ ]:
variance_target = np.zeros([NTEST, 1])
mean_compute = shash.mean(mu, sigma, gamma, tau)

lb = shash.quantile(0.00001 * tf.ones(shape=[NTEST, 1]), mu, sigma, gamma, tau)
ub = shash.quantile(0.99999 * tf.ones(shape=[NTEST, 1]), mu, sigma, gamma, tau)

for i in range(NTEST):
    xx = tf.cast(tf.linspace(lb[i], ub[i], NN), dtype=tf.float32)
    zz = tf.math.square(xx - mean_compute[i])
    ff = shash.prob(xx, mu[i], sigma[i], gamma[i], tau[i])
    variance_target[i] = tf.reduce_mean(zz * ff) * (ub[i] - lb[i])

variance_compute = shash.variance(mu, sigma, gamma, tau)
error = variance_compute - variance_target

print(f"max variance error = {tf.math.reduce_max(error):.6f}")
print(f"min variance error = {tf.math.reduce_min(error):.6f}")

## Test the random number generator

In [ ]:
mu_v = 1.0
sigma_v = 2.0
gamma_v = 1.2
tau_v = 0.8

v = shash.rvs(mu_v, sigma_v, gamma_v, tau_v, size=1000000)
plt.hist(v, bins=50)

print(f"mean error = {np.mean(v) - shash.mean(mu_v, sigma_v, gamma_v, tau_v):.6f}")
print(f"std error  = {np.std(v) - shash.stddev(mu_v, sigma_v, gamma_v, tau_v):.6f}")

## Test the special 3 parameter SHASH

In [ ]:
x = tf.random.uniform(shape=[NTEST, 1], minval=-2, maxval=2)
pr = tf.random.uniform(shape=[NTEST, 1], minval=0.01, maxval=0.99)

mu = tf.random.uniform(shape=[NTEST, 1], minval=-2, maxval=2)
sigma = tf.random.uniform(shape=[NTEST, 1], minval=1, maxval=2)
gamma = tf.random.uniform(shape=[NTEST, 1], minval=-1, maxval=1)
tau = tf.ones((NTEST, 1))

In [ ]:
max_diff = max(
    np.abs(shash.cdf(x, mu, sigma, gamma, tau) - shash.cdf(x, mu, sigma, gamma))
)
print(max_diff)

max_diff = max(
    np.abs(
        shash.log_prob(x, mu, sigma, gamma, tau) - shash.log_prob(x, mu, sigma, gamma)
    )
)
print(max_diff)

max_diff = max(np.abs(shash.mean(mu, sigma, gamma, tau) - shash.mean(mu, sigma, gamma)))
print(max_diff)

max_diff = max(
    np.abs(shash.median(mu, sigma, gamma, tau) - shash.median(mu, sigma, gamma))
)
print(max_diff)

max_diff = max(
    np.abs(shash.prob(x, mu, sigma, gamma, tau) - shash.prob(x, mu, sigma, gamma))
)
print(max_diff)

max_diff = max(
    np.abs(
        shash.quantile(pr, mu, sigma, gamma, tau) - shash.quantile(pr, mu, sigma, gamma)
    )
)
print(max_diff)

max_diff = max(
    np.abs(shash.stddev(mu, sigma, gamma, tau) - shash.stddev(mu, sigma, gamma))
)
print(max_diff)

max_diff = max(
    np.abs(shash.variance(mu, sigma, gamma, tau) - shash.variance(mu, sigma, gamma))
)
print(max_diff)